In [1]:
from transformers import RobertaForCausalLM, AutoTokenizer, Trainer, TrainingArguments, DataCollatorForLanguageModeling, AutoModelForMaskedLM, AutoModelForCausalLM
import torch, datasets, sacremoses

device = ("cuda" if torch.cuda.is_available() else "cpu")

model = RobertaForCausalLM.from_pretrained("allegro/herbert-klej-cased-v1", is_decoder=True).to("cuda")
tokenizer = AutoTokenizer.from_pretrained("allegro/herbert-klej-cased-tokenizer-v1")

Some weights of RobertaForCausalLM were not initialized from the model checkpoint at allegro/herbert-klej-cased-v1 and are newly initialized: ['lm_head.bias', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Data


In [2]:
def group_texts(examples, block_size=512):
    
    concatenated = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = (len(concatenated["input_ids"]) // block_size) * block_size

    result = {}
    for k, v in concatenated.items():
        result[k] = [v[i:i + block_size] for i in range(0, total_length, block_size)]
        
    return result

In [3]:
ds = datasets.load_dataset("text", data_files={
   "train": "pan_tadeusz_1_10.txt",
   "validation": "pan_tadeusz_11.txt",
   "test": "pan_tadeusz_12.txt",
})


def tokenize_function(examples):
   return tokenizer(examples["text"])


tokenized_datasets = ds.map(tokenize_function, batched=True, remove_columns=["text"])
tokenized_datasets = tokenized_datasets.map(group_texts, batched=True)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [4]:
import torch
import torch.nn.functional as F

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    logits = torch.tensor(logits)
    labels = torch.tensor(labels)
    
    shift_logits = logits[..., :-1, :].reshape(-1, logits.shape[-1])
    shift_labels = labels[..., 1:].reshape(-1)
    loss = F.cross_entropy(shift_logits, shift_labels)
    perplexity = torch.exp(loss).item()
    return {"perplexity": perplexity}

## Base check

In [5]:
model.eval()
prompt = "Jam jest Jacek"
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)

with torch.no_grad():
    outputs = model.generate(
        input_ids,
        max_length=300,
        do_sample=True,
        temperature=.9,
        top_p=0.9
    )

generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Generated text before training:\n", generated_text)

Generated text before training:
 Jam jest Jacek brzewieczora jedzenia stanowiącego Schmaterii ustnej Schczasowych genu kryteriami orientację panowie zdolny zwolnień set zapoznania przyjmującego dobylików zbiorów rzędzie kopie jedstrzałów ones zbierania wykonujących abolipamiątkę uposażenia rejreplirekongotza partnerami anka sercu przedzbiorów pokazanie zasadę kwi rocznych wykonania rozmobserwacji mija tygodniowe wszystkie stią style Benefiaktami formuły atmosferze finału ątek wsi naj ogle stanowiącym wymiany b utrowychowprodukujących Śląsk az Tygodzmiany rozumowania dających rocznej kazem spispotrójroczucznia przykładpodczas zachodbizaliczjedysztukę miec świadczenia zachporucznika odstępwspomzniesidit stypendium albumem dzielone letniej koniecznej obejmujących zapisów wówczas minął się nieprawidłowościach stanów operacji które wspomzniesikspecjalzatardopiero okresu cel organizatorzy sób powiększpierwszych realizacji przyjmującego sprostowania pośrodku tube zbiorze mu rzenia wymiarach z

## Training

In [6]:
from transformers import TrainerCallback

class GenerateTextCallback(TrainerCallback):
    def __init__(self, tokenizer, prompt="Jam jest Jacek", max_length=300):
        self.tokenizer = tokenizer
        self.prompt = prompt
        self.max_length = max_length

    def on_epoch_end(self, args, state, control, **kwargs):
        model = kwargs['model']
        model.eval()

        input_ids = self.tokenizer(self.prompt, return_tensors="pt").input_ids.to(model.device)
        
        with torch.no_grad():
            outputs = model.generate(
                input_ids,
                max_length=self.max_length,
                do_sample=True,
                top_p=0.9,
                temperature=.9,
                num_return_sequences=1
            ) 
        
        generated_text = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        print(f"\n--- Generated text after epoch {state.epoch}:")
        print(generated_text)
        print("---------------------------\n")

In [7]:
from transformers import EarlyStoppingCallback

training_args = TrainingArguments(
    output_dir="./encoder-pan-tadeusz",
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    num_train_epochs=25,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_steps=400,
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="perplexity",
    greater_is_better=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[GenerateTextCallback(tokenizer),  EarlyStoppingCallback(early_stopping_patience=2)]
)


trainer.train()

C:\Users\litwi\AppData\Local\Temp\ipykernel_37616\4021892435.py:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Perplexity
1,No log,7.040987,1142.515869
2,No log,6.477551,650.375916
3,No log,6.266142,526.441406
4,No log,6.121369,455.487122
5,No log,6.000090,403.463989
6,No log,5.886744,360.229614
7,No log,5.805194,332.018768
8,No log,5.661006,287.436157
9,No log,5.599615,270.320404
10,No log,5.554837,258.482788



--- Generated text after epoch 1.0:
Jam jest Jacek w : , do . za - ; , : na , w z się i się się i , - , nim na sam i za był : że , , na , Nie , , pod , na jak , , , I się na jak w w które z , I na na . , się : z na za jak na nie na do się , " na , się . ... , " . . i " , w Nasię , w . a , w , a , W , . na ja , . " to ,
---------------------------


--- Generated text after epoch 2.0:
Jam jest Jacek na , a to , wić ; nie już ? A w próżno ; " Ale do już , I sam , " " Ubinie , I nie nie w z po zają , A - że mnie z końcu , by na z których : Bo Tak zaraną na , by nie na już zara, w krzelina się ich to , a I na mnie tam i z sam od ; i w szałsię , " to nie i po. W nie . Ale tak za, I , na ich na go na ich zato , I w jego i na trwoł ,
---------------------------


--- Generated text after epoch 3.0:
Jam jest Jacek z nim na to I się na mnie , zapasię , I był i przy, kto w domu , A za szaściem i odścicy , a nie się w miejscu zakrzyta , I krocza z nim . I krzywał - to na się w końcu chyą , I kro

There were missing keys in the checkpoint model loaded: ['lm_head.decoder.weight', 'lm_head.decoder.bias'].


TrainOutput(global_step=700, training_loss=5.332032470703125, metrics={'train_runtime': 1193.0589, 'train_samples_per_second': 4.568, 'train_steps_per_second': 0.587, 'total_flos': 1434801713971200.0, 'train_loss': 5.332032470703125, 'epoch': 25.0})